In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from 'recursive-set';

In [ ]:
import { RegExp as MyRegExp, RegExp2NFA, NFA } from "./03-RegExp-2-NFA";
import { parse } from "./RegExp-Parser";
import { nfa2dfa, DFA } from "./01-NFA-2-DFA";
import { nfa2dot, dfa2dot, dfa2string } from "./FSM-2-Dot";

In [ ]:
function showSet<T>(s: RecursiveSet<T>): string {
    return `{${Array.from(s).map(String).sort().join(", ")}}`;
}

This notebook contains a test for the function `toNFA` implemented in the notebook `03-Regexp-2-NFA.ts`. In order to write regular expressions conveniently, we need the parser implemented in `RegExp-Parser`.


If the regular expression $r$ that is defined below has been written in the style of the lecture notes, it would read:
$$ (a \cdot b + b \cdot a)^* $$

In [ ]:
const r = parse('(ab+ba)*');
r;

In [ ]:
const sigma = new RecursiveSet("a", "b");

In [ ]:
const converter = new RegExp2NFA(sigma);

In [ ]:
const nfa = converter.toNFA(r);

In [ ]:
console.log(`States: ${showSet(nfa.Q)}`);
console.log(`Start: ${nfa.q0}`);
console.log(`Final: ${showSet(nfa.A)}`);

In [ ]:
console.log("Transitions:");
for (const [key, targets] of nfa.delta) {
    console.log(`  (${key}) -> ${showSet(targets)}`);
}

In [ ]:
const dotNFA = nfa2dot(nfa);

In [ ]:
const viz = await instance();
display.html(viz.renderString(dotNFA, { format: "svg" }));

After having constructed a non-deterministic <span style="font-variant:small-caps;">Fsm</span> for the regular expression, we convert it into a deterministic <span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
const dfa = nfa2dfa(nfa);

In [ ]:
dfa2string(dfa);

In [ ]:
const { dot: dotDFA, statesToNames } = dfa2dot(dfa);
display.html(viz.renderString(dotDFA, { format: "svg" }));

Note that the resulting DFA is not minimal, since we can identify some states.

We can inspect the mapping from DFA states (sets of NFA states) to their names:


In [ ]:
statesToNames;